# Описание задачи
Последний этап проекта посвящен работе непосредственно с ценовыми данными, собранными из онлайн-магазинов.

В процессе работы вы:


*   улучшите свои навыки работы с датафреймами (и
прочими структурами) в Python
*   напишете собственную user-written функцию, осуществляющую фильтрацию сырых данных
*   займетесь визуализацией данных


Подробнее о данных:

Файл prices.csv содержит данные по ценам товаров,которые вы отнесли к выбранным вами категориям на предыдущем этапе.
Названия колонок соответствуют web_price_id идентифицированных товаров, в колонке date_observe – даты (начиная с первого дня публичного существования базы до сегодняшнего).  Для каждого дня в каждой из колонок зафиксирована цена товара, наблюдавшаяся на сайте, где он размещен. Важной особенностью базы является то, что в дни, когда бот просматривает сайт и обнаруживает, что цена не изменилась по сравнению с предыдущей, он не делает запись в базу. Соответственно, в данных появляется NA. Вы можете использовать [эти](https://stackoverflow.com/questions/27905295/how-to-replace-nans-by-preceding-or-next-values-in-pandas-dataframe) методы для заполнения таких пропусков.

Файл prices_rosstat.csv содержит ценовые данные Росстата на ежемесячном уровне.

**Последовательность действий на данном этапе:**


1. Вам нужно разделить общий csv-файл на отдельные датафреймы, согласно выбранным категориям.
2. Имея сырые данные, мы могли бы прямо сейчас рассчитать среднюю цену в категории товаров для каждого дня и получить некую меру изменения цен.

Однако в них существует ряд неизбежных проблем:

a.	Из-за особенностей базы в наблюдениях неизбежно есть пропуски – дни, когда цена не менялась

b.	В данных могут быть неправдоподобно большие/маленькие значения цен. Это может быть ошибка парсинга, базы или самого интернет-магазина, но в любом случае это исказит расчет средней цены

c.	Товар может «умереть»: после какой-то даты в нем может не быть ни одного наблюдения до сегодняшнего дня

d.	На товар может распространяться врéменная скидка. И, хотя методология ИПЦ предполагает учет в индексах цен в том числе и скидок, для других целей бывает желательно скидки и распродажи из расчета исключать. Под временной скидкой давайте понимать следующую ситуацию: цена товара падает, сохраняется на этом уровне менее месяца и возвращается на тот же / более высокий уровень.

3. Вам нужно предложить алгоритм фильтрации ценовых котировок для дальнейшего расчета индекса цен (желательно, чтобы это была функция def <function_name>: …, которая бы проходилась по всем колонкам датафрейма (кроме даты) и обрабатывала их). Алгоритм должен подразумевать решение как минимум двух из проблем, обозначенных выше, хотя, возможно, у вас возникнут дополнительные идеи.

4. После проведенной обработки данных рассчитайте для каждого дня (data_observe) среднюю цену товаров, принадлежащих одной группе. Агрегируйте результат до месячного уровня (найдите среднее для каждого месяца). Результатом будет временной ряд месячной периодичности для каждой из анализируемых вами категорий (3-5 штук).

5.	Изобразите результат расчета на графике (библиотека matplotlib). По горизонтальной оси – время (месяцы), по вертикальной – рассчитанная среднемесячная цена.
Ваша задача – добавить ценовые данные Росстата на тот же график таким образом, чтобы можно было сопоставить ваш расчет с официальной статистической информацией. Фильтровать данные из датасета Росстата достаточно по первой колонке – вашей товарной группе.




## Working with DataFrames

**My categories from the previous part of this project:**

1.Мороженое сливочное, кг

2.Зефир, пастила, кг

3.Футболка детская, шт

4.Джемпер женский, шт






In [ ]:
# download the necessary files from MyDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import math
import matplotlib as plt
!pip install plotly==4.2.1
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 9.8 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=cf20929790df51826ca50a89c73195a5e477a24f038de4afabd46240b99b7767
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
df = pd.read_csv('prices.csv')
df.head(10)

,date_observe,15675,15680,15685,15695,15696,15697,15737,15738,15765,...,11876064,11876065,11876066,11876073,11876074,11876081,11876089,11876091,11876097,11876098
1,2020-06-29,699.0,599.0,699.0,699.0,899.0,599.0,339.0,339.0,799.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,699.0,NaN,NaN,NaN,NaN,NaN,NaN,339.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,599.0,NaN,NaN,NaN,NaN,NaN,339.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-08-24,NaN,599.0,NaN,NaN,NaN,NaN,NaN,339.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-09-11,NaN,599.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-09-16,NaN,599.0,NaN,NaN,NaN,NaN,NaN,339.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-10-14,NaN,599.0,NaN,NaN,NaN,599.0,NaN,NaN,769.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2020-11-02,NaN,599.0,NaN,NaN,909.0,NaN,339.0,335.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_result = pd.read_excel('files_combined_Колмакова.xlsx')
df_result.head(10)

,web_price_id,product_name,Rosstat_name
0,2785197,Петербургский КондитерЪ Зефир в шоколаде натур...,"Зефир, пастила, кг"
1,8831362,Пастила ВкусВилл Яблочная с брусникой без саха...,"Зефир, пастила, кг"
2,11106866,Jess Food Пастила Ассорти фруктов и ягод,"Зефир, пастила, кг"
3,8849293,Пастила PastiLab медовая сливовая 250г,"Зефир, пастила, кг"
4,8831422,Зефир ВкусВилл ванильный 80г,"Зефир, пастила, кг"
5,8674353,Magic Гель для душа Флорентийский зефир,"Зефир, пастила, кг"
6,201788,Массивная доска Бамбук Зефир лак 1 981 м2,"Зефир, пастила, кг"
7,7631611,NaturPastil Пастила яблоко,"Зефир, пастила, кг"
8,2979685,Яблочный зефир Живые снеки классический 60 г,"Зефир, пастила, кг"
9,2983116,Зефир в шоколаде Шармэль классический 250 г,"Зефир, пастила, кг"


In [ ]:
list1=list(df_result['web_price_id'][df_result['Rosstat_name']=='Мороженое сливочное, кг'].astype(str))


In [ ]:
ice_cream=df.loc[:, df.columns.isin(list1)] # now create specific dataframes for every category out of the four
ice_cream

,25228,25232,25265,25266,25269,25277,27163,27197,28544,39665,...,11659052,11659053,11659054,11659059,11683425,11798882,11801410,11814927,11815091,11819112
1,749.0,2199.0,799.0,899.0,1149.0,949.0,199.0,329.0,839.0,359.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,899.0,1149.0,NaN,NaN,NaN,NaN,NaN,599.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,619.0,2139.0,NaN,NaN,969.0,899.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,689.0,889.0,NaN,NaN,NaN,NaN,929.0,419.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,629.3,1.0,NaN,1.0,NaN,NaN,NaN,389.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
list2=list(df_result['web_price_id'][df_result['Rosstat_name']=='Зефир, пастила, кг'].astype(str))

In [ ]:
zephyr=df.loc[:, df.columns.isin(list2)]
zephyr

,47991,72142,133528,133546,133621,133639,133686,133814,133825,133848,...,11820430,11820479,11820490,11835700,11843462,11843483,11843484,11843513,11850233,11850262
1,16499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,128.87,NaN,NaN,135.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
list3=list(df_result['web_price_id'][df_result['Rosstat_name']=='Футболка детская, шт'].astype(str))

In [ ]:
t_shirts=df.loc[:, df.columns.isin(list3)]
t_shirts

,15675,15680,15685,15695,15696,15697,15737,15738,15765,15877,...,11876064,11876065,11876066,11876073,11876074,11876081,11876089,11876091,11876097,11876098
1,699.0,599.0,699.0,699.0,899.0,599.0,339.0,339.0,799.0,999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,699.0,NaN,NaN,NaN,NaN,NaN,NaN,339.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,599.0,NaN,NaN,NaN,NaN,NaN,339.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
list4=list(df_result['web_price_id'][df_result['Rosstat_name']=='Джемпер женский, шт'].astype(str))

In [ ]:
sweaters=df.loc[:, df.columns.isin(list4)]
sweaters

,292980,387298,415059,415060,415407,415411,415487,415640,415642,415643,...,11849202,11849205,11849210,11849214,11852736,11852834,11852835,11853983,11858305,11860032
1,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ice_cream.insert(0, 'date_observe', df['date_observe'])
ice_cream

,date_observe,25228,25232,25265,25266,25269,25277,27163,27197,28544,...,11659052,11659053,11659054,11659059,11683425,11798882,11801410,11814927,11815091,11819112
1,2020-06-29,749.0,2199.0,799.0,899.0,1149.0,949.0,199.0,329.0,839.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,899.0,1149.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,619.0,2139.0,NaN,NaN,969.0,899.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,689.0,889.0,NaN,NaN,NaN,NaN,929.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,629.3,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,2021-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2022-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
zephyr.insert(0, 'date_observe', df['date_observe'])
zephyr

,date_observe,47991,72142,133528,133546,133621,133639,133686,133814,133825,...,11820430,11820479,11820490,11835700,11843462,11843483,11843484,11843513,11850233,11850262
1,2020-06-29,16499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,128.87,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,2021-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2022-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
t_shirts.insert(0, 'date_observe', df['date_observe'])
t_shirts

,date_observe,15675,15680,15685,15695,15696,15697,15737,15738,15765,...,11876064,11876065,11876066,11876073,11876074,11876081,11876089,11876091,11876097,11876098
1,2020-06-29,699.0,599.0,699.0,699.0,899.0,599.0,339.0,339.0,799.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,699.0,NaN,NaN,NaN,NaN,NaN,NaN,339.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,599.0,NaN,NaN,NaN,NaN,NaN,339.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,2021-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2022-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sweaters.insert(0, 'date_observe', df['date_observe'])
sweaters

,date_observe,292980,387298,415059,415060,415407,415411,415487,415640,415642,...,11849202,11849205,11849210,11849214,11852736,11852834,11852835,11853983,11858305,11860032
1,2020-06-29,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,2021-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2022-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def filt(jim): # creating the function to avoid mistakes connected with collecting data about prices
  a=list(jim.columns)
  a=a[1:]
  for elim in a:
    c=jim[elim]
    dat=list(c)
    gaps=[]
    for elem in range(len(dat)):
      if math.isnan(dat[elem])== True:
        gaps.append(elem)
    if sum(gaps[len(gaps)-200:])<129900: #checking that there was a change at least once in 200 estimations otherwise the good shouldn't be taken in count
      if 457 not in gaps: #checking that there was the first estimation, 458-the number of the row with the first estimation
        for elem in gaps:
          dat[elem]=dat[elem-1] #filling the gaps with the price of the good if the price hasn't changed this date
        jim[elim]=dat
        if dat[elem-1]<=dat[elem]*3:
          dat[elem]=dat[elem-1]
      else: # in case the good hasn't been noticed in time of the first estimations, this number is changed into the
        s=0 #average price of the goods througout the period of estimations
        e=0
        for elem in dat:
          if math.isnan(elem)== False:
            s+=elem
            e+=1
        dat[457]=round((s/e),2) #the average price is rounded to the 2 first digits after the dot
        for elem in gaps:
          dat[elem]=dat[elem-1]
        if dat[elem-1]<=dat[elem]*3:
          dat[elem]=dat[elem-1]
        jim[elim]=dat
    else:
      jim=jim.drop(columns=[elim]) # the 'dead' good is taken out of count as it hasn't been noticed in period of 200 latest estimations

  jim.to_csv(r'D:\downloads\jim.csv', sep=',', index = False, encoding='utf-8')

  return jim

In [ ]:
ice_cream_filted=filt(ice_cream)
ice_cream_filted

,date_observe,2982197,2982214,2982226,2982247,2982249,2982288,2982316,2982318,2982334,...,3421308,3421381,3432303,4074458,4691362,5013403,7695193,7695221,7695576,11099747
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,72.9,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0
756,2022-01-17,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,72.9,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0
757,2021-10-09,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,72.9,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0
758,2022-07-05,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,72.9,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0


In [ ]:
zephyr_filted=filt(zephyr)
zephyr_filted

,date_observe,2784616,2784797,2785188,2785321,2785365,2947202,3051138,3191870,3219651,...,10410656,10411927,10411996,10781710,10781758,10781810,10782289,10782329,10786304,10920821
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0
756,2022-01-17,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0
757,2021-10-09,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0
758,2022-07-05,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0


In [ ]:
sweaters_filted=filt(sweaters)
sweaters_filted


,date_observe,456168,1093913,1094095,1094129,1668190,1673756,3535080,3542821,3634170,...,11111105,11114545,11122149,11130480,11140605,11166539,11199647,11302653,11302656,11339364
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,5472.0,4508.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,5472.0,4508.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0
756,2022-01-17,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0
757,2021-10-09,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0
758,2022-07-05,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0


In [ ]:
t_shirts_filted=filt(t_shirts)


In [ ]:
t_shirts_filted

In [ ]:
dates_list=['июнь 2020','июль 2020','август 2020','сентябрь 2020','октябрь.2020','ноябрь.2020','декабрь.2020',
            'январь.2021','февраль.2021','март.2021','апрель.2021','май.2021','июнь.2021','июль.2021',
            'август.2021','сентябрь.2021','октябрь.2021','ноябрь.2021','декабрь.2021','январь.2022','февраль.2022',
            'март.2022','апрель.2022','май.2022','июнь.2022','июль.2022','август.2022']

In [ ]:
ice_cream_filted['mean'] = ice_cream_filted.mean(axis=1)
ice_cream_filted

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,date_observe,2982197,2982214,2982226,2982247,2982249,2982288,2982316,2982318,2982334,...,3421381,3432303,4074458,4691362,5013403,7695193,7695221,7695576,11099747,mean
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0,122.333548
756,2022-01-17,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0,122.333548
757,2021-10-09,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0,122.333548
758,2022-07-05,52.99,235.99,52.99,198.99,59.99,89.99,71.99,59.99,35.99,...,65.9,89.9,178.99,429.99,72.9,119.99,89.99,345.99,517.0,122.333548


In [ ]:
ice_cream_filted[['date_observe','mean']]

,date_observe,mean
1,2020-06-29,NaN
2,2020-08-08,NaN
3,2020-08-18,NaN
4,2020-09-07,NaN
5,2020-07-29,NaN
...,...,...
755,2021-10-27,122.333548
756,2022-01-17,122.333548
757,2021-10-09,122.333548
758,2022-07-05,122.333548


In [ ]:
 ice_cream_f = ice_cream_filted.groupby(pd.PeriodIndex(ice_cream_filted['date_observe'], freq="M"))['mean'].mean().reset_index()
 ice_cream_f

,date_observe,mean
0,2020-06,162.624462
1,2020-07,152.409540
2,2020-08,139.516445
3,2020-09,138.893746
4,2020-10,128.742509
5,2020-11,138.285315
6,2020-12,139.033451
7,2021-01,140.426114
8,2021-02,143.334721
9,2021-03,168.946254


In [ ]:
my_list =  ice_cream_f["mean"].tolist()
my_list

[162.6244623655914,
 152.4095395179577,
 139.51644478122208,
 138.89374584605045,
 128.74250882786643,
 138.2853152706417,
 139.033450528512,
 140.42611377542858,
 143.33472113981125,
 168.94625390218522,
 165.759294265233,
 150.9174960332174,
 150.22585993196438,
 142.36825509825732,
 148.2065954408383,
 148.10802277936992,
 155.57475256429314,
 149.80402752440983,
 152.4403853558628,
 147.67452546251903,
 166.30298149717598,
 161.21742043010755,
 163.22961715703653,
 172.88249581545634,
 179.17539488320355,
 175.5841505376344,
 168.12315249266862]

In [ ]:
sct=go.Scatter(x=dates_list,y=my_list, marker={'color':'salmon'})
lay=go.Layout(title='Изменение цены на мороженое по месяцам', xaxis={'title':'месяц'}, yaxis={'title':'цена'})
fig=go.Figure(data=sct,layout=lay)
fig.show()

In [ ]:
zephyr_filted=filt(zephyr)
zephyr_filted

,date_observe,2784616,2784797,2785188,2785321,2785365,2947202,3051138,3191870,3219651,...,10410656,10411927,10411996,10781710,10781758,10781810,10782289,10782329,10786304,10920821
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0
756,2022-01-17,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0
757,2021-10-09,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0
758,2022-07-05,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,396.0,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0


In [ ]:
zephyr_filted['mean'] = zephyr_filted.mean(axis=1)
zephyr_filted

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,date_observe,2784616,2784797,2785188,2785321,2785365,2947202,3051138,3191870,3219651,...,10411927,10411996,10781710,10781758,10781810,10782289,10782329,10786304,10920821,mean
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.000000
5,2020-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0,354.618182
756,2022-01-17,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0,354.618182
757,2021-10-09,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0,354.618182
758,2022-07-05,599.0,326.0,559.0,326.0,174.0,420.0,191.0,363.0,204.0,...,366.0,389.0,174.0,563.0,413.0,725.0,427.0,559.0,427.0,354.618182


In [ ]:
zephyr_filted[['date_observe','mean']]

,date_observe,mean
1,2020-06-29,NaN
2,2020-08-08,NaN
3,2020-08-18,NaN
4,2020-09-07,189.000000
5,2020-07-29,189.000000
...,...,...
755,2021-10-27,354.618182
756,2022-01-17,354.618182
757,2021-10-09,354.618182
758,2022-07-05,354.618182


In [ ]:
zephyr_f = zephyr_filted.groupby(pd.PeriodIndex(ice_cream_filted['date_observe'], freq="M"))['mean'].mean().reset_index()
zephyr_f

,date_observe,mean
0,2020-06,345.721970
1,2020-07,319.230674
2,2020-08,286.549887
3,2020-09,278.630107
4,2020-10,291.654804
5,2020-11,282.353328
6,2020-12,309.540237
7,2021-01,291.883321
8,2021-02,305.549463
9,2021-03,339.231378


In [ ]:
my_list2 =  zephyr_f["mean"].tolist()
my_list2

[345.72196969696967,
 319.2306739008535,
 286.549886586181,
 278.6301066472906,
 291.65480427490553,
 282.35332751867895,
 309.5402374880167,
 291.8833208502326,
 305.5494630164581,
 339.23137829912025,
 337.65444444444444,
 311.4628219800634,
 319.37922432522555,
 311.2714071369244,
 333.07000833886684,
 321.6936592992598,
 334.2921658986175,
 337.23198106491213,
 342.66069745069746,
 352.1851084310062,
 341.55641085181315,
 342.5977917102055,
 345.1941608391608,
 364.5989467084639,
 360.3187042842215,
 347.90171717171717,
 350.6117079889807]

In [ ]:
sct=go.Scatter(x=dates_list,y=my_list2, marker={'color':'salmon'})
lay=go.Layout(title='Изменение цены на зефир по месяцам', xaxis={'title':'месяц'}, yaxis={'title':'цена'})
fig=go.Figure(data=sct,layout=lay)
fig.show()

In [ ]:
sweaters_filted=filt(sweaters)
sweaters_filted

,date_observe,456168,1093913,1094095,1094129,1668190,1673756,3535080,3542821,3634170,...,11111105,11114545,11122149,11130480,11140605,11166539,11199647,11302653,11302656,11339364
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,5472.0,4508.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-29,NaN,NaN,5472.0,4508.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0
756,2022-01-17,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0
757,2021-10-09,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0
758,2022-07-05,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0


In [ ]:
sweaters_filted['mean'] = sweaters_filted.mean(axis=1)
sweaters_filted

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,date_observe,456168,1093913,1094095,1094129,1668190,1673756,3535080,3542821,3634170,...,11114545,11122149,11130480,11140605,11166539,11199647,11302653,11302656,11339364,mean
1,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-07,NaN,NaN,5472.0,4508.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4990.000000
5,2020-07-29,NaN,NaN,5472.0,4508.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4990.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-10-27,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0,1805.392857
756,2022-01-17,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0,1805.392857
757,2021-10-09,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0,1805.392857
758,2022-07-05,3057.0,2070.0,3120.0,2352.0,3596.0,3236.0,1749.0,1469.0,3596.0,...,363.0,1350.0,1874.0,2900.0,1404.0,3250.0,363.0,363.0,2050.0,1805.392857


In [ ]:
sweaters_filted[['date_observe','mean']]

,date_observe,mean
1,2020-06-29,NaN
2,2020-08-08,NaN
3,2020-08-18,NaN
4,2020-09-07,4990.000000
5,2020-07-29,4990.000000
...,...,...
755,2021-10-27,1805.392857
756,2022-01-17,1805.392857
757,2021-10-09,1805.392857
758,2022-07-05,1805.392857


In [ ]:
sweaters_f = sweaters_filted.groupby(pd.PeriodIndex(ice_cream_filted['date_observe'], freq="M"))['mean'].mean().reset_index()
sweaters_f

,date_observe,mean
0,2020-06,1811.868056
1,2020-07,1933.418329
2,2020-08,2163.708723
3,2020-09,2351.821848
4,2020-10,2133.142686
5,2020-11,2110.521839
6,2020-12,1958.803266
7,2021-01,1963.714384
8,2021-02,1959.793746
9,2021-03,1707.194124


In [ ]:
my_list3 =  sweaters_f["mean"].tolist()
my_list3

[1811.8680555555557,
 1933.4183285291715,
 2163.7087227280426,
 2351.821847788051,
 2133.1426864613413,
 2110.5218389935535,
 1958.8032662902401,
 1963.7143836379616,
 1959.79374580561,
 1707.194124423963,
 1699.3957671957673,
 1748.9204321332456,
 1796.9657695987282,
 1747.3864416746858,
 1810.2401712317078,
 1792.0408625709874,
 1739.7819359793866,
 1805.2014950775033,
 1797.6379003063214,
 1802.652162014231,
 1747.36221340388,
 1753.0842923280425,
 1754.2116147741147,
 1785.839127968374,
 1752.3574849480021,
 1678.7644841269841,
 1712.3541666666665]

In [ ]:
sct=go.Scatter(x=dates_list,y=my_list3, marker={'color':'salmon'})
lay=go.Layout(title='Изменение цены на свитеры женские по месяцам', xaxis={'title':'месяц'}, yaxis={'title':'цена'})
fig=go.Figure(data=sct,layout=lay)
fig.show()

In [ ]:
t_shirts_filted=filt(t_shirts)
t_shirts_filted

In [ ]:
prices_rosstat = pd.read_csv('prices_rosstat.csv')

In [ ]:
grouped=list(prices_rosstat.groupby("KIPC"))
grouped

[('Джемпер женский, шт.',
                        KIPC                 OKATO  OKATO_id     EI  period  \
  2     Джемпер женский, шт.  Российская Федерация       643  рубль       1   
  5     Джемпер женский, шт.  Российская Федерация       643  рубль       2   
  8     Джемпер женский, шт.  Российская Федерация       643  рубль       3   
  11    Джемпер женский, шт.  Российская Федерация       643  рубль       4   
  14    Джемпер женский, шт.  Российская Федерация       643  рубль       5   
  ...                    ...                   ...       ...    ...     ...   
  1019  Джемпер женский, шт.  Российская Федерация       643  рубль       3   
  1023  Джемпер женский, шт.  Российская Федерация       643  рубль       4   
  1027  Джемпер женский, шт.  Российская Федерация       643  рубль       5   
  1031  Джемпер женский, шт.  Российская Федерация       643  рубль       6   
  1035  Джемпер женский, шт.  Российская Федерация       643  рубль       7   
  
        year    value  

In [ ]:
sweaters_rosstat=grouped[0][1].drop(columns=['OKATO','OKATO_id','year','period','EI']) #removing useless columns
zephyr_rosstat=grouped[1][1].drop(columns=['OKATO','OKATO_id','year','period','EI'])
ice_cream_rosstat=grouped[2][1].drop(columns=['OKATO','OKATO_id','year','period','EI'])

In [ ]:
sweaters_rosstat=sweaters_rosstat.loc[sweaters_rosstat['date'].isin(['2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01',
'2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01',
'2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01','2022-07-01'])]

In [ ]:
ice_cream_rosstat=ice_cream_rosstat.loc[ice_cream_rosstat['date'].isin(['2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01',
'2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01',
'2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01','2022-07-01'])]

In [ ]:
zephyr_rosstat=zephyr_rosstat.loc[zephyr_rosstat['date'].isin(['2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01',
'2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01',
'2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01','2022-07-01'])]

In [ ]:
sweaters_rosstat_prices=list(sweaters_rosstat['value']) #creating the list with prices of goods
ice_cream_rosstat_prices=list(ice_cream_rosstat['value'])
zephyr_rosstat_prices=list(zephyr_rosstat['value'])

In [ ]:
trace1 = go.Scatter(
    x=dates_list,
    y=sweaters_rosstat_prices,
    name='rosstat price',
    marker={'color':'rgb(34,163,192)'})
sct=go.Scatter(x=dates_list, y=my_list3, name='project_price', marker={'color':'salmon'})
lay=go.Layout(title='Изменение цены на свитеры женские по месяцам', xaxis={'title':'месяц'}, yaxis={'title':'цена'})
fig=go.Figure(data=[sct,trace1],layout=lay)
fig.show()

In [ ]:
trace1 = go.Scatter(
    x=dates_list,
    y=ice_cream_rosstat_prices,
    name='rosstat price',
    marker={'color':'rgb(34,163,192)'})
sct=go.Scatter(x=dates_list,y=my_list, name='project_price', marker={'color':'salmon'})
lay=go.Layout(title='Изменение цены на мороженое по месяцам', xaxis={'title':'месяц'}, yaxis={'title':'цена'})
fig=go.Figure(data=[sct,trace1],layout=lay)
fig.show()

In [ ]:
trace1 = go.Scatter(
    x=dates_list,
    y=zephyr_rosstat_prices,
    name='rosstat price',
    marker={'color':'rgb(34,163,192)'})
sct=go.Scatter(x=dates_list,y=my_list2, name='project_price', marker={'color':'salmon'})
lay=go.Layout(title='Изменение цены зефир, пастила по месяцам', xaxis={'title':'месяц'}, yaxis={'title':'цена'})
fig=go.Figure(data=[sct,trace1],layout=lay)
fig.show()